In [ ]:

# A very simple Flask Hello World app for you to get started with...

from flask import Flask, request, render_template_string

app = Flask(__name__)

# -------------------------
# Simple meal database
# -------------------------
MEALS = {
    "pre": [
        "Oatmeal with banana and honey",
        "Turkey sandwich on white bread",
        "Rice with scrambled eggs",
    ],
    "during": [
        "Banana and water",
        "Orange slices and pretzels",
        "Sports drink and granola bar",
    ],
    "post": [
        "Chicken and rice bowl",
        "Protein smoothie with fruit",
        "Greek yogurt with berries and granola",
    ],
}

# -------------------------
# Home page with form
# -------------------------
HTML_FORM = """
<!doctype html>
<title>Pickleball Fuel Finder</title>
<h1>🏓 Pickleball Meal Recommendations</h1>

<form method="post">
  <label>How long are you playing?</label><br>
  <select name="duration">
    <option value="short">Less than 1 hour</option>
    <option value="medium">1–3 hours</option>
    <option value="long">Tournament / 3+ hours</option>
  </select><br><br>

  <label>When do you want food for?</label><br>
  <select name="timing">
    <option value="pre">Before playing</option>
    <option value="during">Between games</option>
    <option value="post">After playing</option>
  </select><br><br>

  <input type="submit" value="Get Recommendations">
</form>

{% if recs %}
  <h2>Recommended Meals:</h2>
  <ul>
    {% for meal in recs %}
      <li>{{ meal }}</li>
    {% endfor %}
  </ul>
{% endif %}
"""

# -------------------------
# Main route
# -------------------------
@app.route("/", methods=["GET", "POST"])
def home():
    recs = None

    if request.method == "POST":
        timing = request.form.get("timing")
        recs = MEALS.get(timing, [])

    return render_template_string(HTML_FORM, recs=recs)


# -------------------------
# For local testing only
# -------------------------
if __name__ == "__main__":
    app.run(debug=True)
